In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
index_ent=pd.read_csv("../input/all-csv/index_df (1).csv")

In [3]:
index_ent.head()

,index_text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,text
0,"NAME INDEX TO VOLUME 7 OF LETTERS SENT , JULY ...","['Abbott', 'Lyman', 'Adams', 'John Q', 'Allen'...",[],[],"NAME INDEX TO VOLUME 7 OF LETTERS SENT , JULY ..."
1,"Boyd , J. F. Col. 19 , 41 Boyden , James 116 B...","['Boyd', 'J. F. Col.', 'Boyden', 'James', 'Bra...",[],['Henry Corp'],"Boyd , J. F. Col. 19 , 41 Boyden , James 116 B..."
2,"Chur , J. F. Capt . 139 Cilley , Clinton A. Ma...","['Chur', 'J. F.', 'Cilley', 'Clinton A. Maj', ...",[],['Citizens'],"Chur , J. F. Capt . 139 Cilley , Clinton A. Ma..."
3,"Elijah 176 Eliza 172 Emery , Charles Capt . 3 ...","['Elijah', 'Eliza', 'Emery', 'Charles', 'Edwin...",[],[],"Elijah 176 Eliza 172 Emery , Charles Capt . 3 ..."
4,"Hale , Capt . 51 Hambrich , P.R . Capt . 9 Han...","['Hale', 'Hambrich', 'Hand', 'Harkness', 'E.J'...",['N.C'],[],"Hale , Capt . 51 Hambrich , P.R . Capt . 9 Han..."


In [4]:
index_text=index_ent["index_text"]
index_names=index_ent["PERSON-nerenglarge"]
index_loc=index_ent["LOC-nerenglarge"]
index_org=index_ent["ORG-nerenglarge"]

In [5]:
index_text[150]

'1865 . London Manger 14 . London Sarah E 16 . Lippett John E 18 . Lippett W.H 18 . 23 . 33 . Laslo J.G 19 . Lehman R.F 20 . 44 . Lecraft Mary 21 . Lazarus Julia 22 . Lippett W. 23 . 33 . Lecraft Benjamin 36 . 59 . Lewis K.H 30 . Lewis W.F 30 . Levy T.J. 31 . Lane H.B 37 . Leach J A 40 . 45 . Leavy J.P 50 . Lord Fredk 52 . Littlejohn W.H 53 . Leigh J.W 66 . Low T.B 66 .'

In [6]:
index_names[150]

"['London Sarah E', 'Lippett John E', 'Lippett W.H', 'Lehman R.F', 'Lazarus Julia', 'Lippett W', 'Lecraft Benjamin', 'Lewis K.H', 'Lewis W.F', 'Levy T.J', 'Lane H.B', 'Leach J A', 'Leavy J.P', 'Lord Fredk', 'Littlejohn W.H', 'Leigh J.W', 'Low T.B']"

In [7]:
index_text[1500]

'298 WHEN RECEIVED . NAME OF WRITER . DATE AND PURPORT OF LETTER . 299 WHEN RECEIVED . NAME OF WRITER . DATE AND PURPORT OF LETTER . 1.3.70 . 1 Missions , Presby Home . Rev . E. F. Hatfield Secretary per . W. T. M. New York City. , Dec. 30th 1869 . States reply , Mr. Swan , teacher due three months salary , 60 withheld case regard purchase of books investigated . 1.8.70 . 2 Moss , R. C. Teacher . Bladensburg , Md . Jany 6th 1870 . States 81 70100 still due School house lot : names of trustees Darius Bowman , Uriah Wallace , Davis Greenleaf , Cyrus Johnston . 1 , 13 , 70 3 Morris Susan , Teacher . Poolsville , Montgomery Co. Md . January 3 , 1870 . Requests check amount due sent Mr. Adams . 1 , 17 , 70 4 Miller Lewis E. Teacher Lewistown , Frederick Co. Md . Pock Mark Jany 15th 70 , States opening of school Lewistown , forwards compliance instructions Baltimore Association . Will report , instructed report made . 1 , 18 , 70 , 5 Moses Rebecca , Teacher Point of Rocks , Fdk . Co. Md . P.

In [8]:
index_names[1500]

"['E. F. Hatfield', 'Swan', 'Darius Bowman', 'Uriah Wallace', 'Davis Greenleaf', 'Cyrus Johnston', 'Morris Susan', 'Miller Lewis E.', 'Pock Mark', 'Moses Rebecca', 'Moss R', 'C.']"

In [9]:
index_text[5000]

'Jenny 90 . Johnson Richard 10 Jones Andrew 12 Johnston Thos . P. A.N.U . 16 , 20 , 75 Jordan Alice 37 , 53 Jordan Vina 51 , 54 , 55 Johnston Tho.P . A.N.U . 65 , 74 95 , 169 , 199 , 200 , 204 , 205 , 281 Johnston Hugh 107 , 146 Jane 129 Jennings Mr 184 , 185 Johnston David 219 Johnson Leah 228 Jeremia Parthe 253'

In [10]:
index_names[5000]

"['Johnson Richard', 'Jones Andrew', 'Johnston Thos', 'Jordan Alice', 'Jordan Vina', 'Johnston Hugh', 'Jane', 'Jennings Mr', 'Johnston David', 'Johnson Leah', 'Jeremia Parthe']"

In [11]:
index_names

0       ['Abbott', 'Lyman', 'Adams', 'John Q', 'Allen'...
1       ['Boyd', 'J. F. Col.', 'Boyden', 'James', 'Bra...
2       ['Chur', 'J. F.', 'Cilley', 'Clinton A. Maj', ...
3       ['Elijah', 'Eliza', 'Emery', 'Charles', 'Edwin...
4       ['Hale', 'Hambrich', 'Hand', 'Harkness', 'E.J'...
                              ...                        
5417                                                   []
5418    ['Wm . F. Richardson', 'Hattie M. Richardson',...
5419    ['Hattie L. Newcomb', 'Mutch Opelika', 'Thomas...
5420    ['Mutch Opelika', 'Thomas H. Whitly', 'Emma A ...
5421    ['James Nichols Stevenson', 'O . M. Waring', '...
Name: PERSON-nerenglarge, Length: 5422, dtype: object

In [12]:
PL=[]
empty=0
for i in index_names:
    for j in i:
        PL.append(j)
for i in index_names:
    if i==[]:
        empty=empty+1
len(PL), empty

(1298703, 0)

# Names from Index are Perfect 

****

# Order ontonotes

In [13]:
order_ent=pd.read_csv("../input/all-csv/to_compare_result_order.csv")

In [14]:
order_ent.head()

,order_text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,postDate-ontonotes,standard_date-ontonotes,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Other_name-nerenglish
0,R G. R.R . THIS ORDER NOT TRANSFERABLE . U.S. ...,"['E . Cranford', 'Lewis B. Parsons', 'Major Kn...","['Raleigh N.C', 'Warrenton N.C', 'New York']",['Samuel Gale Co .'],"['Aug 6 1866', 'Aug 1866']",[],['67487'],['6-Aug-1866'],['6-August-1866'],"['Cranford', 'Lewis B. Parsons', 'Gaston R.R']","['U.S.', 'Raleigh N.C', 'Warrenton N.C', 'Rale...","['Samuel Gale Co', 'Regiment Freedman', 'Rail ...",[],[],"['Ral', 'Gaston R.R', 'Raleigh N.C.', 'Broadwa...",['Cranford'],[]
1,"Certified Copy of Order , Extract , within tra...",['E. Crawford'],['Raleigh N.C'],[],['Aug 6 1866'],[],[],['6-Aug-1866'],['6-August-1866'],"['Robinson Clinton', 'E. Crawford']",['Raleigh N.C'],[],[],[],[],[],[]
2,"Bureau of Refugees , Freedmen Abandoned Lands ...","['KELLEY', 'EDWARD McPHERSON', 'JONATHAN WORTH']","['Raleigh', 'N.C.', 'United States', 'Alabama'...","['Bureau of Refugees', 'House of Representativ...","['May 8th , 1866', 'May 1 , 1866', '1st']",[],"['2d', 'III']","['8-May-1866', '1-May-1866']","['8-May-1866', '1-May-1866']","['KELLEY', 'EDWARD McPHERSON', 'E. WHITTLESEY'...","['State of North Carolina', 'Raleigh', 'N.C.',...","['Bureau of Refugees , Freedmen Abandoned Land...",[],['JONATHAN WORTH'],"['Raleigh', 'N.C.', 'United States', 'Alabama'...","['Officers Agents of Bureau', 'House of Repres...",[]
3,"HEADQUARTERS DEPARTMENT OF NORTH CAROLINA , Ra...",['Joseph E. Johnston'],"['Raleigh', 'North Carolina']","['HEADQUARTERS DEPARTMENT OF NORTH CAROLINA', ...","['January 12th , 1866']",[],"['4', 'II', 'III']",['12-January-1866'],['12-January-1866'],['Joseph E. Johnston'],"['NORTH CAROLINA', 'Raleigh', 'North Carolina'...",['Government'],[],['Joseph E. Johnston'],"['NORTH CAROLINA', 'Raleigh', 'North Carolina']",[],[]
4,IV . Animals sold counties collected unless wo...,"['J A . Campbell', 'James H Anderson']",['U.S.'],"['Headquarters', 'Government']","['first day of February , proximo']",[],['VI'],[],[],"['J A . Campbell', 'James H Anderson']","['U.S.', 'U.S. C.S', 'C.S']",['Government'],[],['James H Anderson'],['U.S.'],[],['C.S']


In [15]:
order_text=order_ent["order_text"]
order_Per_onto=order_ent["PERSON-ontonotes"]
order_GPE_onto=order_ent["GPE-ontonotes"]
order_ORG_onto=order_ent["ORG-ontonotes"]
order_date_onto=order_ent["standard_date-ontonotes"]
order_loc_onto=order_ent["LOC-ontonotes"]

In [16]:
order_text[1000]

'190 By Order of Col. James V. Bomford Jacob F. Chur . Bvt . Lt. Col. A.A.A.G . Bureau of Refugees Freedmen A.L . Head Qrs . Asst . Commissioner Raleigh N.C. Jan. 16th 1867 Special Order No . 8 Leave of absence twelve days hereby granted Lieut Edgar H. McQuigg V.R.C . Asst Supt Bureau R.F . A.L . Wilmington N.C. By Command of Col. J. V. Bomford Jacob F. Chur . Bvt . Lt. Col. A.A.A . Genl Bureau of Refugees Freedmen c Hd Qrs Asst Commissioner Raleigh N.C. Jan. 18th 1867 Special Order No . 9 In compliance instructions Maj Genl O.O . Howard Commissioner , Bvt . Maj. Thos P. Johnston Chf Qr Mr Bureau State pay trustees of Baptist Female Seminary following amounts rent of rooms used Bureau upon premises known Guion Hotel City of Raleigh recommended board of Officers appointed purpose . For three rooms main building occupied offices of Bureau of R.F . A.L . April 1st 1866 Aug 1st 1866 rate of Twelve dollars per month per room . For four rooms detatched building occupied offices of Bureau of 

In [17]:
order_Per_onto[1000]

"['James V. Bomford', 'J. V. Bomford', 'Manchester W. Weld', 'Justin Hodge', 'M. W. Weld', 'Jacob F. Chur', 'F . A. Seeley']"

In [18]:
order_GPE_onto[1000]

"['Raleigh N.C', 'Raleigh', 'Fayetteville N.C.']"

In [19]:
order_loc_onto[1000]

'[]'

In [20]:
order_date_onto[1000]

"['16-Jan.-1867', '18-Jan.-1867', '1-April-1866', '1-August-1866', '2-April-1866', '31-December-1866', '19-Jan.-1867', '20-Jan.-1867']"

**good**

In [21]:
PL=[]
empty=0
for i in order_Per_onto :
    for j in i:
        PL.append(j)
for i in order_Per_onto:
    if i==[]:
        empty=empty+1
len(PL), empty

(453588, 0)

**very good**

In [22]:
PL=[]
empty=0
for i in order_date_onto :
    for j in i:
        PL.append(j)
for i in order_date_onto:
    if i==[]:
        empty=empty+1
len(PL), empty

(167185, 0)

**uii very good**

In [23]:
PL=[]
empty=0
for i in order_GPE_onto :
    for j in i:
        PL.append(j)
for i in order_GPE_onto:
    if i==[]:
        empty=empty+1
len(PL), empty

(343109, 0)

****

# Order Ner-english

In [24]:
order_ent.head()

,order_text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,postDate-ontonotes,standard_date-ontonotes,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Other_name-nerenglish
0,R G. R.R . THIS ORDER NOT TRANSFERABLE . U.S. ...,"['E . Cranford', 'Lewis B. Parsons', 'Major Kn...","['Raleigh N.C', 'Warrenton N.C', 'New York']",['Samuel Gale Co .'],"['Aug 6 1866', 'Aug 1866']",[],['67487'],['6-Aug-1866'],['6-August-1866'],"['Cranford', 'Lewis B. Parsons', 'Gaston R.R']","['U.S.', 'Raleigh N.C', 'Warrenton N.C', 'Rale...","['Samuel Gale Co', 'Regiment Freedman', 'Rail ...",[],[],"['Ral', 'Gaston R.R', 'Raleigh N.C.', 'Broadwa...",['Cranford'],[]
1,"Certified Copy of Order , Extract , within tra...",['E. Crawford'],['Raleigh N.C'],[],['Aug 6 1866'],[],[],['6-Aug-1866'],['6-August-1866'],"['Robinson Clinton', 'E. Crawford']",['Raleigh N.C'],[],[],[],[],[],[]
2,"Bureau of Refugees , Freedmen Abandoned Lands ...","['KELLEY', 'EDWARD McPHERSON', 'JONATHAN WORTH']","['Raleigh', 'N.C.', 'United States', 'Alabama'...","['Bureau of Refugees', 'House of Representativ...","['May 8th , 1866', 'May 1 , 1866', '1st']",[],"['2d', 'III']","['8-May-1866', '1-May-1866']","['8-May-1866', '1-May-1866']","['KELLEY', 'EDWARD McPHERSON', 'E. WHITTLESEY'...","['State of North Carolina', 'Raleigh', 'N.C.',...","['Bureau of Refugees , Freedmen Abandoned Land...",[],['JONATHAN WORTH'],"['Raleigh', 'N.C.', 'United States', 'Alabama'...","['Officers Agents of Bureau', 'House of Repres...",[]
3,"HEADQUARTERS DEPARTMENT OF NORTH CAROLINA , Ra...",['Joseph E. Johnston'],"['Raleigh', 'North Carolina']","['HEADQUARTERS DEPARTMENT OF NORTH CAROLINA', ...","['January 12th , 1866']",[],"['4', 'II', 'III']",['12-January-1866'],['12-January-1866'],['Joseph E. Johnston'],"['NORTH CAROLINA', 'Raleigh', 'North Carolina'...",['Government'],[],['Joseph E. Johnston'],"['NORTH CAROLINA', 'Raleigh', 'North Carolina']",[],[]
4,IV . Animals sold counties collected unless wo...,"['J A . Campbell', 'James H Anderson']",['U.S.'],"['Headquarters', 'Government']","['first day of February , proximo']",[],['VI'],[],[],"['J A . Campbell', 'James H Anderson']","['U.S.', 'U.S. C.S', 'C.S']",['Government'],[],['James H Anderson'],['U.S.'],[],['C.S']


In [25]:
order_Per_ner=order_ent["PERSON-nerenglish"]
order_ORG_ner=order_ent["ORG-nerenglish"]
order_other_onto=order_ent["Other_name-nerenglish"]
order_loc_ner=order_ent["LOC-nerenglish"]

In [26]:
order_text[2]

'Bureau of Refugees , Freedmen Abandoned Lands , Headquarters Assistant Commissioner , State of North Carolina , Raleigh , N.C. , May 8th , 1866 . Circular No . 4 The attention of Officers Agents of Bureau called following resolution of House of Representatives : THIRTYNINTH CONGRESSFIRST SESSION . Congress of United States . In House of Representatives , May 1 , 1866 . Mr. KELLEY submitted following , adopted : WHEREAS , It reported citizens of Alabama , formal memorial two Houses of Congress , many of people of mountain districts of State suffering want of adequate supplies of food , considerable numbers of died of actual starvation ; therefore , Be resolved , That President requested instruct proper officers of Bureau of Refugees Freedman enquire condition of said districts , districts of late insurgent States , suffering may said exist , relieve people thereof , provide corn seed planting crop sufficient annual supply of family requiring relief . ATTEST : Signed EDWARD McPHERSON , 

In [27]:
order_Per_ner[2]

"['JONATHAN WORTH']"

In [28]:
order_ORG_ner[2]

"['Officers Agents of Bureau', 'House of Representatives', 'Houses of Congress', 'Bureau of Refugees Freedman', 'Freedmens Bureau']"

**names of ner english are catastrophic**

****

# Order Ner english large 